## Exercise 1

We have selected **hate speech detection** as the binary classifier and the **sentiment analysis** as the multiclass classifier.

In [180]:
#Imports 
import pandas as pd 
import csv 
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import TweetTokenizer
from difflib import ndiff


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [45]:
# Data loading 

# hate
hate_train = open("datasets/hate/train_text.txt", "r", encoding="utf-8").read()
hate_test = open("datasets/hate/test_text.txt", "r", encoding="utf-8").read()
hate_mapping = open("datasets/hate/mapping.txt", "r", encoding="utf-8").read()

# sentiment 
sentiment_train = open("datasets/sentiment/train_text.txt", "r", encoding="utf-8").read()
sentiment_test = open("datasets/sentiment/test_text.txt", "r", encoding="utf-8").read()
sentiment_mapping = open("datasets/sentiment/mapping.txt", "r", encoding="utf-8").read()

**Preprocessing** <br>
Using regular expressions, implement a tokeniser to split the input texts into meaningful tokens. Your tokeniser should be a script that takes as input the data as distributed in the dataset and outputs the tokenised text, one output line per input line, with spaces between tokens.

In [189]:
def tokenize_line(line):
    #replacements = ".,:;?!/#%\(\)@$\-…"
    #temp = re.sub(f'([^ ])([{replacements}])', r'\1 \2', line)
    #temp = re.sub(f'([{replacements}])([^ ])', r'\1 \2', temp)
    temp = re.sub(f'([a-zA-Z])([^a-zA-Z ])', r'\1 \2', line)
    temp = re.sub(f'([^a-zA-Z @#])([a-zA-Z])', r'\1 \2', temp)
    return temp

tokenize_line('hej, d/et.te (er) en .test!')

'hej , d / et . te ( er ) en . test !'

1. Use the training set of one of the tasks you’ve chosen to work on. Set aside a part of the training set for evaluating the tokeniser so you don’t have to touch the validation data for that.

In [134]:
hate_train_lists = hate_train.split('\n')
lhate_train_token)ize = hate_train_lists[0:4500]
hate_train_set_aside = hate_train_lists[4500:-1]

2. Start by taking a subset of the training data, look through it and discuss in your group what a good tokenisation of this subset should look like. Then design your initial Python implementation to match this ideal tokenisation, and run it over the portion of the training set that you didn’t hold out for tokeniser evaluation. Keep an eye on infrequently occurring tokens in the output that look like tokenisation errors.


In [161]:
[tokenize_line(line) for line in hate_train_tokenize[0:20]]

['@ user nice new signage . Are you not concerned by Beatlemania - style hysterical crowds crongregating on you … ',
 'A woman who you fucked multiple times saying yo dick small is a compliment you know u hit that spot 😎 ',
 '@ user @ user real talk do you have eyes or were they gouged out by a rapefugee ? ',
 'your girlfriend lookin at me like a groupie in this bitch ! ',
 'Hysterical woman like @ user ',
 'Me flirting - So tell me about your father ... ',
 'The Philippine Catholic bishops \' work for migrant workers should focus on families who are " paying the great ... ',
 "I AM NOT GOING AFTER YOUR EX BF YOU LIEING SACK OF SHIT ! I ' m done with you dude that ' s why I dumped your ass cause your a lieing 😂😡 bitch ",
 'When cuffin season is finally over ',
 'Send home migrants not in need of protection , Peter Dutton tells UN , HEY DUTTON HOW ABOUT THE ONES THAT HAVE STAYED AND NOT LEFT THE COUNTRY WHEN THEY SHOULD OVERSTAYERS ? WHY DONT YOU GO AND ROUND ALL THEM UP ?  ',
 'Cory Bo

3. Once you’re done, compare your tokeniser’s output with the baseline tokenisation
you get from the social media tokeniser in the NLTK library (nltk.tokenize.TweetTokenizer), using the data you’ve set aside for this purpose.

In [190]:
#TweetTokenizer(hate_train)
tknzr = TweetTokenizer()
tokens_homemade = [tokenize_line(hate_train_tokenize[i].strip()) for i in range(20)]
tokens_nltk = [' '.join(tknzr.tokenize(hate_train_tokenize[i])) for i in range(20)]

In [191]:
print(tokens_homemade)
print(tokens_nltk)
[i for i in ndiff(tokens_homemade, tokens_nltk)]

['@user nice new signage . Are you not concerned by Beatlemania - style hysterical crowds crongregating on you …', 'A woman who you fucked multiple times saying yo dick small is a compliment you know u hit that spot 😎', '@user @user real talk do you have eyes or were they gouged out by a rapefugee ?', 'your girlfriend lookin at me like a groupie in this bitch !', 'Hysterical woman like @user', 'Me flirting - So tell me about your father ...', 'The Philippine Catholic bishops \' work for migrant workers should focus on families who are " paying the great ...', "I AM NOT GOING AFTER YOUR EX BF YOU LIEING SACK OF SHIT ! I ' m done with you dude that ' s why I dumped your ass cause your a lieing 😂😡 bitch", 'When cuffin season is finally over', 'Send home migrants not in need of protection , Peter Dutton tells UN , HEY DUTTON HOW ABOUT THE ONES THAT HAVE STAYED AND NOT LEFT THE COUNTRY WHEN THEY SHOULD OVERSTAYERS ? WHY DONT YOU GO AND ROUND ALL THEM UP ?', 'Cory Booker and Kamala Harris co

['  @user nice new signage . Are you not concerned by Beatlemania - style hysterical crowds crongregating on you …',
 '  A woman who you fucked multiple times saying yo dick small is a compliment you know u hit that spot 😎',
 '  @user @user real talk do you have eyes or were they gouged out by a rapefugee ?',
 '  your girlfriend lookin at me like a groupie in this bitch !',
 '  Hysterical woman like @user',
 '  Me flirting - So tell me about your father ...',
 '  The Philippine Catholic bishops \' work for migrant workers should focus on families who are " paying the great ...',
 "- I AM NOT GOING AFTER YOUR EX BF YOU LIEING SACK OF SHIT ! I ' m done with you dude that ' s why I dumped your ass cause your a lieing 😂😡 bitch",
 '?                                                            - -                         - -\n',
 "+ I AM NOT GOING AFTER YOUR EX BF YOU LIEING SACK OF SHIT ! I'm done with you dude that's why I dumped your ass cause your a lieing 😂 😡 bitch",
 '?                 

4. Consider using the difflib package in Python, or the diff utility in the Unix shell, to compare the output of the two tokenisers efficiently.

## Exercise 2

Exercise: N-gram Language Modelling <br>

In this exercise, you will get familiar with n-gram language models using the
nltk library.<br>

You can work with the corpora from the two tasks you selected for your project.
Additionally, you can download a dataset of news editorials collected and
distributed by the organisers of the Conference on Machine Translation from
here:
http://data.statmt.org/news-commentary/v16/training-monolingual/news-commentary-v16.en.gz <br>

This is a sizeable data set (albeit still quite small by the standards of the
NLP community), and if you find that your computer becomes intolerably slow or
runs into memory problems, you can just use a subset of the corpus for the
exercises.<br>


1. Remove a subset of about 5000 sentences from the news commentary dataset to
be used for evaluation. For the TweetEval datasets, you can use the standard
training/validation split. <br>



In [62]:
# Loading in the data
news_commentary = open("news-commentary-v16.en", "r", encoding="utf-8").read()

CellOutputSizeError: Cell output is too big.

In [82]:
news_commentary = nltk.sent_tokenize(news_commentary)


In [121]:
d = {'sentences': news_commentary[0:5000]}

news_commentary[0:5000]
pd_news5k = pd.DataFrame(d)
pd_news5k

,sentences
0,"$10,000 Gold?"
1,SAN FRANCISCO – It has never been easy to have...
2,"Lately, with gold prices up more than 300% ove..."
3,"Just last December, fellow economists Martin F..."
4,Wouldn’t you know it?
...,...
4995,"If we have learned anything since 2001, it is ..."
4996,Address to the Kurdish Nation\nERBIL – By hist...
4997,And in spirit I am a member of a very old peop...
4998,That is an experience that the Kurds are now p...


2. Load and tokenise your datasets so that, for each of the corpora, you get a
list of sentences, each sentence represented by a list of tokens. Use the same
tokeniser for all datasets. <br>


In [125]:
tokenized_text = [nltk.word_tokenize(i) for i in pd_news5k["sentences"]]
pd_news5k["words"] = tokenized_text
amount_words = lambda x: len(x)
pd_news5k['num_words'] = pd_news5k['words'].apply(amount_words)

pd_news5k

,sentences,words,num_words
0,"$10,000 Gold?","[$, 10,000, Gold, ?]",4
1,SAN FRANCISCO – It has never been easy to have...,"[SAN, FRANCISCO, –, It, has, never, been, easy...",19
2,"Lately, with gold prices up more than 300% ove...","[Lately, ,, with, gold, prices, up, more, than...",21
3,"Just last December, fellow economists Martin F...","[Just, last, December, ,, fellow, economists, ...",28
4,Wouldn’t you know it?,"[Wouldn, ’, t, you, know, it, ?]",7
...,...,...,...
4995,"If we have learned anything since 2001, it is ...","[If, we, have, learned, anything, since, 2001,...",19
4996,Address to the Kurdish Nation\nERBIL – By hist...,"[Address, to, the, Kurdish, Nation, ERBIL, –, ...",32
4997,And in spirit I am a member of a very old peop...,"[And, in, spirit, I, am, a, member, of, a, ver...",41
4998,That is an experience that the Kurds are now p...,"[That, is, an, experience, that, the, Kurds, a...",13


3. Follow the instructions at
http://www.nltk.org/api/nltk.lm.html#module-nltk.lm
to train maximum-likelihood language models of varying orders (e.g., n=2..6) for
each of your corpora. Make a note of the size of the n-gram lists for each
n-gram order. You might also plot them. <br>

IMPORTANT: We will want to compare perplexities across different corpora. This
only produces meaningful results if all the models use exactly the same
vocabulary. To ensure that, create a vocabulary from the largest of your
datasets and use it for ALL the corpora. <br>

In [ ]:
vocabulary = list()


4. Use the lm.generate function to generate some example text from each of your
models and compare. <br>

5. Use lm.score and lm.logscore to calculate the scores of a couple of n-grams
you find in the texts with the different models. Try 1 or 2 n-grams with all
function words and 1 or 2 n-grams that contain specific content words or names.

6. Use the lm.perplexity function to compute the perplexity of the validation
sets from each dataset with each of the language models and compare. Keep in
mind that LOWER perplexity is BETTER.



7. Repeat the steps above with another language model implementation, such as
Laplace, WittenBellInterpolated or KneserNeyInterpolated.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5235cf77-d347-437f-a336-60ef81766ce7' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>